In [99]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg 
import scipy
import pandas as pd
import copy
import seaborn as sns
import pickle
from scipy import sparse

In [45]:
data_file = "./country_hsproduct2digit_year.csv"
Data=pd.read_csv(data_file,low_memory=False)
#give id for country 
def giveId(name):
    Dict={}
    n=0
    for c in  Data[name]:
        if c not in Dict:
            Dict[c]=n
            n+=1    
    return Dict

CountryId=giveId("location_name_short_en")
CountryCodeId=giveId("location_code")
ProductCodeId=giveId("hs_product_code")
ProductId=giveId("hs_product_name_short_en")

In [66]:
N,M=len(CountryCodeId),len(ProductCodeId)

In [81]:
MatrixD={}
for index in Data.index:
    year=Data.loc[index]['year']
    if year not in MatrixD:
        MatrixD[year]=np.zeros((N,M))
    ev = Data.loc[index]["export_value"]
    c = Data.loc[index]["location_code"]
    p = Data.loc[index]["hs_product_code"]
    MatrixD[year][CountryCodeId[c]][ProductCodeId[p]]=ev

In [83]:
with open("MatrixD.pkl",'wb')as f:
    pickle.dump(MatrixD,f)

In [161]:
R>1

/home/liujing/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater
  """Entry point for launching an IPython kernel.


matrix([[False, False, False, ..., False, False,  True],
        [False, False, False, ..., False, False, False],
        [False, False,  True, ..., False, False, False],
        ...,
        [ True,  True, False, ..., False, False, False],
        [False,  True,  True, ..., False, False, False],
        [False, False, False, ..., False, False, False]])

In [208]:
def RCA(Xcp):
    """
    Xcp is a numpy.matrix type
    --|steel|oil|chip|bean|clothes
    --|--|--|--|--|--
    USA|1|1|1|1|0
    China|1|0|0|1|1
    Viet Nam|0|0|0|1|1
    """
    if type(Xcp) is not np.matrix:
        if type(Xcp) is np.array or list:
            Xcp = np.matrix(Xcp)
        else:
            raise ValueError('Xcp must be matrix')
    else:
        pass
    B = Xcp.sum(1)*Xcp.sum(0) 
    Sum = Xcp.sum()
    return Xcp*Sum/B
    
def fliterRCA(R):
    M = R>1
    return M.astype(float)


def Get_eci_pci(M):
    d = M.sum(1).T.tolist()[0]
    u = M.sum(0).tolist()[0]
    D = np.diag([1.0/i if i>0 else 0. for i in d])
    U = np.diag([1.0/i if i>0 else 0. for i in u])
    mcp1 = D * M
    mcp2 = M * U
    
    Mcc = mcp1 * mcp2.T
    Mpp = mcp2.T * mcp1
    eigvals, eigvecs = np.linalg.eig(Mpp)
    eigvecs = np.real(eigvecs)
    # Get eigenvector corresponding to second largest eigenvalue
    eig_index = eigvals.argsort()[-2]
    kp = eigvecs[:, eig_index]
    kc = mcp1 @ kp
    s1 = np.sign(np.corrcoef(M.sum(1).reshape(-1), kc.reshape(-1))[0, 1])
    eci = s1 * kc
    pci = s1 * kp  
    return eci.T.tolist()[0],pci.T.tolist()[0]

def Get_eci_pci_sparse(M):
    d = M.sum(1).T.tolist()[0]
    u = M.sum(0).tolist()[0]
    nd,nu = len(d),len(u)
    D1 = scipy.sparse.csc_matrix(([1.0/i if i>0 else 0. for i in d], (range(nd), range(nd))),shape=(nd,nd))
    U1 = scipy.sparse.csc_matrix(([1.0/i if i>0 else 0. for i in u], (range(nu), range(nu))),shape=(nu,nu))
    M = sparse.csc_matrix(M)
    mcp1 = D1 * M
    mcp2 = M * U1
    Mcc = mcp1 @ mcp2.T
    Mpp = mcp2.T @ mcp1
    A=scipy.sparse.csc_matrix(Mpp)
    eigvals, eigvecs = scipy.sparse.linalg.eigs(A, k=2)
    eigvecs = np.real(eigvecs)
    # Get eigenvector corresponding to second largest eigenvalue
    eig_index = eigvals.argsort()[-2]
    kp = eigvecs[:, eig_index]
    kc = mcp1 @ kp
    s1 = np.sign(np.corrcoef(M.sum(1).reshape(-1), kc.reshape(-1))[0, 1])
    eci = s1 * kc
    pci = s1 * kp
    return eci,pci

def Get_z_score(x):
    x=np.array(x)
    std=np.std(x)
    mean=np.mean(x)
    return (x-mean)/std

In [250]:
c = MatrixD[1995]
R = RCA(c)
M = fliterRCA(R)
eci,pci = Get_eci_pci(np.matrix(M))

/home/liujing/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in true_divide
/home/liujing/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
/home/liujing/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in greater


In [212]:
Get_z_score(pci)

array([-0.72126308, -0.34339136, -1.16835553,  0.13200883, -0.87178221,
       -0.88787332, -1.10629223, -1.10942161, -1.62093656, -0.95646448,
       -0.76928976, -1.34394039, -1.12705631, -1.135165  , -0.92962389,
       -0.69602306, -0.99192977, -0.6936576 , -0.11351231, -0.57266281,
       -0.59476154, -0.55844643, -0.93922899, -0.67414876, -0.4166958 ,
       -1.55429746, -1.46323413, -0.32848758,  0.61608846,  0.8480846 ,
       -0.60680007,  1.65793062, -0.31023481,  0.16068982,  1.08149574,
       -0.01273841,  1.71548025,  1.17097571,  1.47944455,  0.48323147,
       -0.82281482,  0.4358478 , -0.12403756, -0.9463549 ,  0.01433409,
       -0.72622839,  0.08596344,  0.84865751,  0.83470305,  0.44479095,
        0.11322817, -1.03049504, -0.15222095,  1.21865977,  0.68044649,
        0.72063143, -0.41820157,  1.24675418,  1.48082347,  1.29678812,
       -0.29467336, -0.47236149, -0.08604713,  0.34053629,  0.12985454,
        1.41645139,  0.02156039,  0.60836099,  0.47767747,  0.88

In [213]:
cdata.mcp_t

AttributeError: 'DataFrame' object has no attribute 'mcp_t'

In [171]:
from ecomplexity import ecomplexity
from ecomplexity import proximity
from ecomplexity import ComplexityData
# Calculate complexity
trade_cols = {'time':'year', 'loc':'location_code', 'prod':'hs_product_code', 'val':'export_value'}
#cdata = ecomplexity(data, trade_cols)

# # Calculate proximity matrix
# prox_df = proximity(data, trade_cols)

In [215]:
import numpy as np
import pandas as pd
import warnings
import sys
from functools import wraps
import time
import datetime

class ComplexityData(object):
    """Calculate complexity and other related results
    Args:
        data: pandas dataframe containing production / trade data.
            Including variables indicating time, location, product and value
        cols_input: dict of column names for time, location, product and value.
            Example: {'time':'year', 'loc':'origin', 'prod':'hs92', 'val':'export_val'}
        val_errors_flag: {'coerce','ignore','raise'}. Passed to pd.to_numeric
            *default* coerce.
    Attributes:
        data: clean data with standardized column names
    """

    def __init__(self, data, cols_input, val_errors_flag):
        self.data = data.copy()

        # Standardize column names based on input
        self.rename_cols(cols_input)

        # Clean data to handle NA's and such
        self.clean_data(val_errors_flag)

    def rename_cols(self, cols_input):
        """Standardize column names"""
        cols_map_inv = {v: k for k, v in cols_input.items()}
        self.data = self.data.rename(columns=cols_map_inv)
        self.data = self.data[['time', 'loc', 'prod', 'val']]

    def clean_data(self, val_errors_flag_input):
        """Clean data to remove non-numeric values, handle NA's and duplicates"""
        # Make sure values are numeric
        self.data.val = pd.to_numeric(
            self.data.val, errors=val_errors_flag_input)
        self.data.set_index(['time', 'loc', 'prod'], inplace=True)
        if self.data.val.isnull().values.any():
            warnings.warn('NaN value(s) present, coercing to zero(es)')
            self.data.val.fillna(0, inplace=True)

        # Remove duplicates
        dups = self.data.index.duplicated()
        if dups.sum() > 0:
            warnings.warn(
                'Duplicate values exist, keeping the first occurrence')
            self.data = self.data[~self.data.index.duplicated()]

    def create_full_df(self, t):
        """Rectangularize, but remove rows with diversity or ubiquity zero
        Rows with zero diversity / ubiquity lead to dividebyzero errors and
        incorrect values during normzalization
        """
        self.data_t = self.data.loc[t].copy()
        diversity_check = self.data_t.reset_index().groupby(
            ['loc'])['val'].sum().reset_index()
        ubiquity_check = self.data_t.reset_index().groupby(
            ['prod'])['val'].sum().reset_index()
        diversity_check = diversity_check[diversity_check.val != 0]
        ubiquity_check = ubiquity_check[ubiquity_check.val != 0]
        self.data_t = self.data_t.reset_index()
        self.data_t = self.data_t.merge(
            diversity_check[['loc']], on='loc', how='right')
        self.data_t = self.data_t.merge(
            ubiquity_check[['prod']], on='prod', how='right')
        self.data_t.set_index(['loc','prod'], inplace=True)
        data_index = pd.MultiIndex.from_product(
            self.data_t.index.levels, names=self.data_t.index.names)
        self.data_t = self.data_t.reindex(data_index, fill_value=0)

    def calculate_rca(self):
        """Calculate RCA"""
        # Convert data into numpy array
        loc_n_vals = len(self.data_t.index.levels[0])
        prod_n_vals = len(self.data_t.index.levels[1])
        data_np = self.data_t.values.reshape((loc_n_vals, prod_n_vals))

        # Calculate RCA, disable dividebyzero errors
        with np.errstate(divide='ignore', invalid='ignore'):
            num = (data_np / np.nansum(data_np, axis=1)[:, np.newaxis])
            loc_total = np.nansum(data_np, axis=0)[np.newaxis, :]
            world_total = np.nansum(loc_total, axis=1)[:, np.newaxis]
            den = loc_total / world_total
            self.rca_t = num / den

    def calculate_rpop(self, pop, t):
        """Calculate RPOP"""
        # After constructing df with all combinations, convert data into ndarray
        loc_n_vals = len(self.data_t.index.levels[0])
        prod_n_vals = len(self.data_t.index.levels[1])
        data_np = self.data_t.values.reshape(
            (loc_n_vals, prod_n_vals))

        pop.columns = ['time', 'loc', 'pop']
        pop_t = pop[pop.time == t].copy()
        pop_t = pop_t.drop(columns="time")
        pop_t = pop_t.reset_index(drop=True).set_index('loc')
        pop_index = self.data_t.index.unique('loc')
        pop_t = pop_t.reindex(pop_index)
        pop_t = pop_t.values
        # print(pop_t.shape, data_np.shape)

        num = data_np / pop_t
        # print("Num done. Num shape {}".format(num.shape))
        loc_total = np.nansum(data_np, axis=0)[np.newaxis, :]
        world_pop_total = np.nansum(pop_t)

        den = loc_total / world_pop_total
        # print("Den done. Den shape {}".format(den.shape))
        rpop = num / den
        self.rpop_t = rpop

    def calculate_mcp(self, rca_mcp_threshold_input, rpop_mcp_threshold_input,
                           presence_test, pop, t):
        """Calculate MCP based on RCA / RPOP / both"""
        def convert_to_binary(x, threshold):
            x = np.nan_to_num(x)
            x = np.where(x >= threshold, 1, 0)
            return(x)

        if presence_test == "rca":
            self.mcp_t = convert_to_binary(self.rca_t, rca_mcp_threshold_input)

        elif presence_test == "rpop":
            self.calculate_rpop(pop, t)
            self.mcp_t = convert_to_binary(self.rpop_t, rpop_mcp_threshold_input)

        elif presence_test == "both":
            self.calculate_rpop(pop, t)
            self.mcp_t = convert_to_binary(self.rca_t, rca_mcp_threshold_input) + \
                         convert_to_binary(self.rpop_t, rpop_mcp_threshold_input)

    def calculate_manual_mcp(self):
        """If pre-computed MCP supplied, check validity and reshape"""
        # Test to see if indeed MCP
        if np.any(~np.isin(self.data_t.values, [0, 1])):
            error_val = self.data_t.values[~np.isin(
                self.data_t.values, [0, 1])].flat[0]
            raise ValueError(
                "Manually supplied MCP column contains values other than 0 or 1 - Val: {}".format(error_val))

        # Convert data into numpy array
        loc_n_vals = len(self.data_t.index.levels[0])
        prod_n_vals = len(self.data_t.index.levels[1])
        data_np = self.data_t.values.reshape(
            (loc_n_vals, prod_n_vals))

        self.mcp_t = data_np

In [218]:
cdata=ComplexityData(data,trade_cols, 'coerce')

In [232]:
MatrixD[1999][0:]

array([[1.65300000e+04, 5.79300000e+03, 2.45309000e+05, ...,
        8.24636880e+07, 1.54798880e+08, 1.65921788e+06],
       [0.00000000e+00, 3.65270000e+04, 4.59000000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.98420000e+04, 2.29340160e+07, ...,
        4.70000000e+07, 9.02500000e+07, 1.57300000e+07],
       ...,
       [5.39740900e+06, 3.40466040e+07, 1.79849800e+06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.02553300e+06, 1.07444310e+07, 1.11377835e+09, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.73285050e+07, 4.76599800e+07, 7.48597570e+07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [233]:
cdata.calculate_rca()

AttributeError: 'ComplexityData' object has no attribute 'data_t'

In [234]:
cdata.data.index.unique("time")

Int64Index([1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
            2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
           dtype='int64', name='time')

In [235]:
cdata.create_full_df(1995)

In [236]:
cdata.calculate_rca()

In [244]:
presence_test="rca"
val_errors_flag='coerce'
rca_mcp_threshold=1
rpop_mcp_threshold=1
pop=None
cdata.calculate_mcp(rca_mcp_threshold, rpop_mcp_threshold, presence_test, pop, 1995)

In [252]:
cdata.rca_t

array([[9.70606686e-03, 3.64167054e-03, 0.00000000e+00, ...,
        9.77592507e-01, 7.14778921e+00, 0.00000000e+00],
       [1.29280043e-01, 0.00000000e+00, 2.61928089e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.12561452e+00, ...,
        2.62034060e-01, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.47862000e-01, 3.13158526e-01, 1.48667781e+00, ...,
        7.76747702e-01, 1.04453707e+00, 0.00000000e+00],
       [1.38559678e-01, 9.72126269e-03, 3.14519144e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.62464452e+00, 4.22631054e+00, 1.05004137e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [254]:
x = np.where(R >= 1, 1, 0)

/home/liujing/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in greater_equal
  """Entry point for launching an IPython kernel.


In [255]:
x

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])